# Organización del número de contratos por municipio

## Limpieza de los datos

<b>Objetivo:</b> eliminar las líneas innecesarias, y organizar los encabezados de los documentos. De esta manera podremos unificar todos los datos posteriormente.

In [1]:
import os

def clean_data(input_folder, output_folder, year):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['Contratos', 'Serie;;;']
    if year == 2023:
        headers = "Serie;Codigo_municipio;Nombre;2022;2023\n"
    else:
        headers = "Serie;Codigo_municipio;Nombre;2020\n"


    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for file in csv_files:
        original_file = os.path.join(input_folder, file)
        clean_file = os.path.join(output_folder, file)
        with open(original_file, 'r') as f_input, open(clean_file, 'w') as f_output:
            # Escribir los headers
            f_output.write(headers) 
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = True
                    if not delete:
                        fields = line.strip().split(';')
                        if year == 2023:
                            f_output.write(';'.join(fields[:3] + fields[-3:]) + '\n')
                        else:
                            f_output.write(';'.join(fields[:3] + fields[-5:-4]) + '\n')
                    # Eliminar líneas posteriores al ser el último municipio
                    if 'Zarzalejo' in line:
                        delete_next = True

clean_data(".\original_data", ".\clean_data", 2023)
clean_data(".\original_data", ".\clean_data_2020", 2020)


## Unión de los datos

<b>Objetivo:</b> unir todos los archivos de la carpeta en uno solo unidos por sector, con sus encabezados correspondientes. Guardar el <i>.csv</i> resultante en la carpeta <i>union_final</i> para poder añadir los datos al archivo final.

In [4]:
import os, csv

def join_data(folder_path, output_path, year):
    # Crear un diccionario para almacenar los datos unidos
    join_data = {}

    # Obtener la lista de archivos en la carpeta
    csv_files = os.listdir(folder_path)

    # Iterar sobre cada archivo en la carpeta
    for file in csv_files:
        if file.endswith('.csv'):
            with open(os.path.join(folder_path, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')

                # Saltar la primera fila que contiene los headers
                next(csv_reader)

                # Identiicar tipo, sector y subtipo de los archivos
                type = file.split('_')[0]

                if 'agricultura' in file:
                    sector = 'primario'
                elif 'construccion' in file or 'industria' in file:
                    sector = 'secundario'
                elif 'servicios' in file:
                    sector = 'terciario'
                else:
                    sector = None

                if sector:
                    subtype = f'{sector}_{file.split(".")[0].split("_")[-1]}'
                else:
                    subtype = file.split('.')[0].split('_')[-1]

                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""
                    
                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in join_data:
                        join_data[(serie, codigo_municipio, nombre)] = {}

                    if year == 2023: 
                    # Obtener los datos de cada año para el intervalo de edad actual
                        for i, value in enumerate(row[3:5], start=2022):
                            if type == 'contratos':
                                key = f"{type}_{subtype}_{i}"
                            else:
                                key = f"contratos_{type}_{subtype}_{i}"
                            if sector == 'secundario' and key in join_data[(serie, codigo_municipio, nombre)]:
                                actual_value = int((join_data[(serie, codigo_municipio, nombre)][key]))
                                join_data[(serie, codigo_municipio, nombre)][key] = str(actual_value + int(value))
                            else:
                                join_data[(serie, codigo_municipio, nombre)][key] = value

                    else:
                        for i, value in enumerate(row[3:4], start=2020):
                            if type == 'contratos':
                                key = f"{type}_{subtype}"
                            else:
                                key = f"contratos_{type}_{subtype}"
                            if sector == 'secundario' and key in join_data[(serie, codigo_municipio, nombre)]:
                                actual_value = int((join_data[(serie, codigo_municipio, nombre)][key]))
                                join_data[(serie, codigo_municipio, nombre)][key] = str(actual_value + int(value))
                            else:
                                join_data[(serie, codigo_municipio, nombre)][key] = value

    # Escribir los datos unidos en un nuevo archivo CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')

        final_keys = []
        for key, values in join_data.items():
            if final_keys == []:
                final_keys.extend(values.keys())
        final_keys = [str(key) for key in final_keys]

        # Escribir los encabezados en el archivo CSV
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *final_keys])

        # Escribir los datos sumados
        for key, values in join_data.items():
            line = [key[0], key[1], key[2]]
            for clave in final_keys:
                line.append(values.get(clave, ''))
            csv_writer.writerow(line)

join_data('./clean_data', '../union_final/data/contratos.csv', 2023)
join_data('./clean_data_2020', '../union_2020/contratos.csv', 2020)
